## Bias Check

Craig Lage - 11-Nov-22


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.daf.butler import Butler

In [ ]:
butler = Butler('/repo/oga', collections=["LATISS/raw/all", "u/cslage/sdf/latiss/bias_test_20221110"])

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=True
isrConfig.doVariance=True
isrConfig.doLinearize=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False


# Adjust these as needed and add as many more as you want

In [ ]:
isrTask = IsrTask(config=isrConfig)

In [ ]:
expId = 2022110300020
exp = butler.get('raw', detector=0, exposure=expId)
biasExp = butler.get('bias', detector=0, exposure=expId)

In [ ]:
isrResult = isrTask.run(exp, bias=biasExp)

In [ ]:
print(np.min(arr), np.max(arr))

In [ ]:
# Now look at the data with matplotlib
from matplotlib.colors import LogNorm

def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

fig, axs = plt.subplots(1,2,figsize=(16,9))

axs[0].set_title(f"Post ISR Image-{expId}")
arr = isrResult.exposure.image.array
img = axs[0].imshow(arr, interpolation='Nearest', cmap='gray')
colorbar(img)
#plt.tight_layout(h_pad=1)

axs[1].set_title("Ystripe, X = 300")
axs[1].plot(isrResult.exposure.image.array[2100:3900,300])
axs[1].set_xlabel("Y coordinate (pixels)")
axs[1].set_ylabel("ADU")
fig.savefig("/home/c/cslage/u/AuxTel/isr/Bias_Test_10Nov22.pdf")

In [ ]:
diff = isrResult.exposure.image.array - isrResultNoBias.exposure.image.array

In [ ]:
plt.plot(np.mean(biasExp.image.array[2100:3900,0:509], axis=0), label="Master bias")
plt.plot(np.mean(isrResult.exposure.image.array[2100:3900,0:509], axis=0), label="ISR with bias")
plt.plot(np.mean(isrResultNoBias.exposure.image.array[2100:3900,0:509], axis=0) +1.0, label="ISR No bias")
plt.legend()
#plt.xlim(0,100)

In [ ]:
isrResult.exposure.image.array[3500,0:20]

In [ ]:
plt.plot(isrResult.exposure.image.array[2100:3900,300])